In [1]:
# Author: Brent Artuch
# Date: 2024-10-07

import sympy as sp
import numpy as np

# Qiskit libraries
from qiskit.quantum_info import Operator, Statevector
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

sp.init_printing()

## Bell States
The collection of all four Bell states is known as the Bell basis; all four of the Bell states represent entanglement between two qubits.
\begin{align*}
\ket{\beta_{00}}=\ket{\Phi^+}=\frac{1}{\sqrt{2}}(\ket{00}+\ket{11})=CNOT(H)\ket{0}\ket{0}\\
\ket{\beta_{10}}=\ket{\Phi^-}=\frac{1}{\sqrt{2}}(\ket{00}-\ket{11})=CNOT(H)\ket{1}\ket{0}\\
\ket{\beta_{01}}=\ket{\Psi^+}=\frac{1}{\sqrt{2}}(\ket{01}+\ket{10})=CNOT(H)\ket{0}\ket{1}\\
\ket{\beta_{11}}=\ket{\Psi^-}=\frac{1}{\sqrt{2}}(\ket{01}-\ket{10})=CNOT(H)\ket{1}\ket{1}\\
\end{align*}
These states can also be derived from the formula:<br>
\begin{align*}
\ket{\beta_{x,y}}=\frac{1}{\sqrt{2}}\left(\ket{0,y}+(-1)^x\ket{1,\neg y}\right)
\end{align*}

In [2]:
# Define |0> and |1>
ket_zero = Statevector([1, 0]) 
ket_one = Statevector([0, 1]) 

# Define H and CNOT gates
H = Operator([[1 / np.sqrt(2), 1 / np.sqrt(2)], [1 / np.sqrt(2), -1 / np.sqrt(2)]])
CX = Operator([
 [1, 0, 0, 0],
 [0, 1, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0]])

# Plus and minus states
ket_plus = ket_zero.evolve(H)
ket_minus = ket_one.evolve(H)

# Evaluate tensor products
ket_plus_zero = ket_plus.tensor(ket_zero)
ket_minus_zero = ket_minus.tensor(ket_zero)
ket_plus_one = ket_plus.tensor(ket_one)
ket_minus_one = ket_minus.tensor(ket_one)

# Apply the CNOT gate
phi_plus = ket_plus_zero.evolve(CX)
phi_minus = ket_minus_zero.evolve(CX)
psi_plus = ket_plus_one.evolve(CX)
psi_minus = ket_minus_one.evolve(CX)

bell_states = [phi_plus,phi_minus,psi_plus,psi_minus]

for state in bell_states:
    display(state.draw("latex"))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

We can express the Bell States as computational basis vectors of 2-qubit systems, or indeed any complex vector at all having entries corresponding to the four classical states of two bits, can be expressed as a linear combination of the four Bell states. For example, let's find $\ket{00}$ in the Bell Basis:<br>
\begin{align*}
\ket{\Phi^+}&=\frac{1}{\sqrt{2}}(\ket{00}+\ket{11})\\
\ket{\Phi^+}&=\frac{1}{\sqrt{2}}\ket{00}+\frac{1}{\sqrt{2}}\ket{11}\\
-1( -\frac{1}{\sqrt{2}}\ket{00}&=\frac{1}{\sqrt{2}}\ket{11}-\ket{\Phi^+})\\
\frac{1}{\sqrt{2}}\ket{00}&=\ket{\Phi^+}-\frac{1}{\sqrt{2}}\ket{11}\\
\sqrt{2}\ket{00}&=\ket{\Phi^+}+\frac{1}{\sqrt{2}}\ket{00}-\frac{1}{\sqrt{2}}\ket{11}\\
\ket{00}&=\ket{\Phi^+}+\ket{\Phi^-}\\
\ket{00}&=\frac{1}{\sqrt{2}}\ket{\Phi^+}+\frac{1}{\sqrt{2}}\ket{\Phi^-}\\
\end{align*}
The four Z-basis in terms of the Bell Basis:<br>
\begin{align*}
\ket{00}&=\frac{1}{\sqrt{2}}\ket{\Phi^+}+\frac{1}{\sqrt{2}}\ket{\Phi^-}\\
\ket{01}&=\frac{1}{\sqrt{2}}\ket{\Psi^+}+\frac{1}{\sqrt{2}}\ket{\Psi^-}\\
\ket{10}&=\frac{1}{\sqrt{2}}\ket{\Psi^+}-\frac{1}{\sqrt{2}}\ket{\Psi^-}\\
\ket{11}&=\frac{1}{\sqrt{2}}\ket{\Phi^+}-\frac{1}{\sqrt{2}}\ket{\Phi^-}\\
\end{align*}

In [3]:
# Circuit representation for formulation of the Bell States
# Define control and target
X = QuantumRegister(1, "X")
Y = QuantumRegister(1, "Y")
# Define clssical bits for measurement results
A = ClassicalRegister(1, "A")
B = ClassicalRegister(1, "B")

# Apply H gate to control and then CNOT 
circuit = QuantumCircuit(X, Y)
circuit.h(X)
circuit.cx(X, Y)

# Measurement
circuit.measure(X, B)
circuit.measure(Y, A)

display(circuit.draw())

CircuitError: "Bit 'Clbit(ClassicalRegister(1, 'B'), 0)' is not in the circuit."